In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print(cifar_train)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [5]:
print(cifar_test)

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [6]:
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=32, shuffle=True)

In [7]:
import matplotlib.pyplot as plt
import torchvision
images, labels = next(iter(trainloader))
print(images.shape, labels.shape)
# grid = torchvision.utils.make_grid(images)
# grid = torch.transpose(grid, [1,2,0])
# plt.imshow(grid)

torch.Size([32, 3, 32, 32]) torch.Size([32])


#代码练习部分，使用pytorch构建网络

In [8]:
class LeNet(nn.Module):
    # 一般在__init__中定义网络需要的操作算子，比如卷积、全连接算子等等
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.fc1 = nn.Linear(4*4*64, 128)
        self.fc2 = nn.Linear(128, 10)

    # 定义了前向传播的运算，只需要像写普通的python算数运算那样就可以了
    def forward(self, x):
        #进行代码练习
        in_size = x.size(0)
        out = self.bn1(self.conv1(x))
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)
        out = self.bn2(self.conv2(out))
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)
        out = self.bn3(self.conv3(out))
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)
        out = out.view(in_size, -1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out, dim=1)
        return out

In [9]:
# 如果你没有GPU，那么可以忽略device相关的代码
# device = torch.device("cuda:0")
# net = LeNet().to(device)

In [15]:
import torch.optim as optim

# CrossEntropyLoss就是我们需要的损失函数
net = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [16]:
net

LeNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [17]:
def evaluate(epoch):
  # 构造测试的dataloader
  dataiter = iter(testloader)
  # 预测正确的数量和总数量
  correct = 0
  total = 0
  # 使用torch.no_grad的话在前向传播中不记录梯度，节省内存
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          #images, labels = images.to(device), labels.to(device)
          # 预测
          outputs = net(images)
          # 我们的网络输出的实际上是个概率分布，去最大概率的哪一项作为预测分类
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('[{}] acc:{}%'.format(epoch, 100 * correct / total))

In [18]:
print("Start Training...")
for epoch in range(1, 40):
    # 用一个变量来记录每100个batch的平均loss
    loss100 = 0.0
    # 使用dataloader操作数据
    for i, data in enumerate(trainloader):
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss100 += loss.item()
        if i % 500 == 499:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch, i + 1, loss100 / 100))
            loss100 = 0.0

    evaluate(epoch)

print("Done Training!")

Start Training...
[Epoch 2, Batch   500] loss: 8.564
[Epoch 2, Batch  1000] loss: 6.575
[Epoch 2, Batch  1500] loss: 6.003
[1] acc:60.24%
[Epoch 3, Batch   500] loss: 5.353
[Epoch 3, Batch  1000] loss: 5.126
[Epoch 3, Batch  1500] loss: 4.962
[2] acc:65.42%
[Epoch 4, Batch   500] loss: 4.597
[Epoch 4, Batch  1000] loss: 4.509
[Epoch 4, Batch  1500] loss: 4.405
[3] acc:67.97%
[Epoch 5, Batch   500] loss: 4.124
[Epoch 5, Batch  1000] loss: 4.089
[Epoch 5, Batch  1500] loss: 4.049
[4] acc:69.85%
[Epoch 6, Batch   500] loss: 3.806
[Epoch 6, Batch  1000] loss: 3.740
[Epoch 6, Batch  1500] loss: 3.829
[5] acc:71.36%
[Epoch 7, Batch   500] loss: 3.489
[Epoch 7, Batch  1000] loss: 3.531
[Epoch 7, Batch  1500] loss: 3.541
[6] acc:72.32%
[Epoch 8, Batch   500] loss: 3.316
[Epoch 8, Batch  1000] loss: 3.327
[Epoch 8, Batch  1500] loss: 3.310
[7] acc:72.79%
[Epoch 9, Batch   500] loss: 3.030
[Epoch 9, Batch  1000] loss: 3.130
[Epoch 9, Batch  1500] loss: 3.190
[8] acc:74.14%
[Epoch 10, Batch   500

In [19]:
# 首先要把梯度清零，不然PyTorch每次计算梯度会累加，不清零的话第二次算的梯度等于第一次加第二次的       
optimizer.zero_grad()
# 计算前向传播的输出
outputs = net(inputs)
# 根据输出计算loss
loss = criterion(outputs, labels)
# 算完loss之后进行反向梯度传播，这个过程之后梯度会记录在变量中
loss.backward()
# 用计算的梯度去做优化
optimizer.step()

In [20]:
# 构造测试的dataloader
dataiter = iter(testloader)
# 预测正确的数量和总数量
correct = 0
total = 0
# 使用torch.no_grad的话在前向传播中不记录梯度，节省内存
with torch.no_grad():
    for data in testloader:
        images, labels = data
        #images, labels = images.to(device), labels.to(device)
        # 预测
        outputs = net(images)
        # 我们的网络输出的实际上是个概率分布，去最大概率的哪一项作为预测分类
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 74 %
